# Anthropic Claude Tool Use (Function Calling) Example

This notebook demonstrates how to use tool use (function calling) with Anthropic Claude in Dinnovos Agent.

In [ ]:
import os
import json
from dinnovos.llms.anthropic import AnthropicLLM

## 1. Configure API Key

In [ ]:
# Configure your Anthropic API key
api_key = os.getenv("ANTHROPIC_API_KEY") or "your-api-key-here"
llm = AnthropicLLM(
    api_key=api_key, 
    model="claude-sonnet-4-20250514"
)

## 2. Define Tools (Functions)

In [ ]:
# Define available tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Bogotá, Colombia"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

print("Tools defined:", json.dumps(tools, indent=2))

## 3. Implement the Functions

In [ ]:
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Mock weather function"""
    return {
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Sunny",
        "humidity": 65
    }

def calculate(expression: str) -> dict:
    """Calculator function"""
    try:
        result = eval(expression)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}

# Dictionary of available functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

---

# 🚀 RECOMMENDED METHOD: call_with_function_execution()

**The easiest way** to use tool use with Claude. Automatically handles the entire cycle.

**Note:** If you get an error about "system: Input should be a valid list", make sure you're using the latest version of the dinnovos library. The fix ensures that the system parameter is only included when there's an actual system message.

## Example 1: Basic Usage (Automatic)

In [ ]:
# One call does it all!
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # See the complete process
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")

## Example 2: Multiple Functions

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Compare the weather in Bogotá and Medellín, and calculate 25 * 4"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Total functions called: {len(result['function_calls'])}")

## Example 3: Silent Mode

In [ ]:
# Without verbose - just get the result
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Calculate (100 + 50) / 3"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)

print("Response:", result['content'])

## Example 4: Force Tool Use with tool_choice='any'

In [ ]:
# Force Claude to use a tool even if it might not be necessary
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Tell me about weather"}],
    tools=tools,
    available_functions=available_functions,
    tool_choice="any",  # Force tool use
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])

## Example 5: View Complete History

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the temperature in Cali in fahrenheit?"}],
    tools=tools,
    available_functions=available_functions
)

print("\n📊 COMPLETE ANALYSIS:")
print(f"\n✅ Response: {result['content']}")
print(f"\n🔄 Iterations performed: {result['iterations']}")
print(f"\n🔧 Functions executed:")
for i, func_call in enumerate(result['function_calls'], 1):
    print(f"\n  {i}. {func_call['name']}")
    print(f"     Args: {func_call['arguments']}")
    print(f"     Result: {func_call['result']}")

---

# Manual Methods (Advanced)

If you need full control over the process, you can use the low-level methods.

## 6. Example: Simple Call with Tools (Manual)

In [ ]:
messages = [
    {"role": "user", "content": "What's the weather in Bogotá?"}
]

response = llm.call_with_tools(messages, tools)
print("Initial response:")
print(json.dumps(response, indent=2))

In [ ]:
# Execute tool calls if they exist
if response["tool_calls"]:
    for tool_call in response["tool_calls"]:
        function_name = tool_call["function"]["name"]
        function_args = json.loads(tool_call["function"]["arguments"])
        
        print(f"\n🔧 Calling function: {function_name}")
        print(f"📋 Arguments: {function_args}")
        
        # Execute the function
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"✅ Function response: {json.dumps(function_response)}")
        
        # Add the function response to messages
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": response["tool_calls"]
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call["id"],
            "name": function_name,
            "content": json.dumps(function_response)
        })
    
    # Get final response
    final_response = llm.call_with_tools(messages, tools)
    print(f"\n💬 Final Claude response: {final_response['content']}")

## 7. Claude-Specific Features

Claude has some unique features for tool use:

- **`tool_choice="any"`**: Forces Claude to use at least one tool
- **`tool_choice="auto"`**: Claude decides whether to use tools (default)
- **Multiple tool calls**: Claude can call multiple tools in parallel
- **Rich context**: Claude maintains excellent context across tool calls